# Q1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

In [ ]:
!pip install pipenv

In [2]:
import pipenv

pipenv.__version__

'2025.0.4'

# Q2

* Use Pipenv to install Scikit-Learn version 1.5.2
* What's the first hash for scikit-learn you get in Pipfile.lock?

In [ ]:
!pipenv install scikit-learn

In [1]:
with open("Pipfile.lock", "r") as f:
    print(f.read())

{
    "_meta": {
        "hash": {
            "sha256": "92d1e56dea01e9015831d1d3b09b4da8a476d29a38caac2c8f50dc2357dad620"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_full_version": "3.12.10",
            "python_version": "3.12"
        },
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "blinker": {
            "hashes": [
                "sha256:b4ce2265a7abece45e7cc896e98dbebe6cead56bcf805a3d23136d145f5445bf",
                "sha256:ba0efaa9080b619ff2f3459d1d500c57bddea4a6b424b60a91141db6fd2f08bc"
            ],
            "markers": "python_version >= '3.9'",
            "version": "==1.9.0"
        },
        "click": {
            "hashes": [
                "sha256:9b9f285302c6e3064f4330c05f05b81945b2a39544279343e6e7c5f27a9baddc",
                "sha256:e7b8232224eba16f4ebe410c25ced

# Q3

* Write a script for loading these models with pickle
* Score this client:

```json
{"job": "management", "duration": 400, "poutcome": "success"}
```

What's the probability that this client will get a subscription? 

In [2]:
import pickle

with open("dv.bin", "rb") as file:
    dv = pickle.load(file)

with open("model1.bin", "rb") as file:
    model = pickle.load(file)

def predict_subscription_prob(customer, dv, model):
    X_customer = dv.transform(customer)
    return model.predict_proba(X_customer)[:, 1]

customer = {"job": "management", "duration": 400, "poutcome": "success"}

predict_subscription_prob(customer, dv, model)

/Users/hyunwoooh/.local/share/virtualenvs/2024-UrG-U2a_/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/hyunwoooh/.local/share/virtualenvs/2024-UrG-U2a_/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([0.75909665])

# Q4

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a subscription?

In [3]:
import requests
url = 'http://localhost:9696/predict'
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json()

{'churn': False, 'churn_probability': 0.33480703475511053}

Install [Docker](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/05-deployment/06-docker.md). 
We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.11.5-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.11.5-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.11.5-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to [`svizor/zoomcamp-model:3.11.5-slim`](https://hub.docker.com/r/svizor/zoomcamp-model).

# Q5

Download the base image `svizor/zoomcamp-model:3.11.5-slim`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

In [ ]:
!docker pull svizor/zoomcamp-model:3.11.5-slim

In [5]:
!docker image ls

REPOSITORY              TAG           IMAGE ID       CREATED         SIZE
predict-churn           latest        b2780c86c8a8   2 hours ago     570MB
svizor/zoomcamp-model   3.11.5-slim   15d61790363f   12 months ago   197MB


# Q6

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```docker
FROM svizor/zoomcamp-model:3.11.5-slim
# add your stuff here
```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with Gunicorn 

After that, you can build your docker image.


Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {"job": "management", "duration": 400, "poutcome": "success"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a subscription now?


In [ ]:
!docker build --platform=linux/amd64 -t predict .

In [ ]:
!docker run -it --rm --platform=linux/amd64 -p 9696:9696 predict:latest

In [7]:
import requests
url = 'http://localhost:9696/predict'
client = {"job": "management", "duration": 400, "poutcome": "success"}
requests.post(url, json=client).json()

{'churn': True, 'churn_probability': 0.7590966516879658}